In [1]:
import os
import pandas as pd
news_train = {}
news_test = {}
news = []
labels = []


directory = "/home/shushant/Desktop/nlp_projects/NepaliNewsCategoryClassification/data/train"
for filename1 in os.listdir(directory):
    for filename in os.listdir(os.path.join(directory,filename1)):
        if filename.endswith('.txt'):
            # try:
            with open(os.path.join(directory,filename1, filename), encoding="utf8", errors='ignore') as f:
                content = f.read()
                news.append(content)
                labels.append(filename1)
                news_train = dict(zip(news,labels))


directory = "/home/shushant/Desktop/nlp_projects/NepaliNewsCategoryClassification/data/test"
for filename1 in os.listdir(directory):
    for filename in os.listdir(os.path.join(directory,filename1)):
        if filename.endswith('.txt'):
            # try:
            with open(os.path.join(directory,filename1, filename), encoding="utf8", errors='ignore') as f:
                content = f.read()
                news.append(content)
                labels.append(filename1)
                news_test = dict(zip(news,labels))


In [2]:
news_df_train = pd.DataFrame(news_train.items(), columns=['news', 'labels'])

In [3]:
news_df_test = pd.DataFrame(news_test.items(), columns=['news', 'labels'])

In [4]:
news_df_train.shape

(12910, 2)

In [5]:
news_df_test.shape

(14356, 2)

In [6]:
news_df = pd.concat([news_df_train,news_df_test])

In [7]:
news_df.shape

(27266, 2)

In [8]:
news_df["labels"].value_counts()

National News         14148
Sports                 4341
Economy                2255
Entertainment          2230
Opinion                1120
Bank                    666
Tourism                 406
World                   402
Blog                    397
Employment              288
Business Interview      269
Technology              208
Auto                    180
Interview               165
Education               161
Literature               30
Name: labels, dtype: int64

In [9]:
news_df

,news,labels
0,"\n﻿कारोबार संवाददाता\nकाठमाडौं, ९ असार\nगैरआवा...",Employment
1,"\n﻿\n४ पुस, काठमाडौं । १८ वर्षमा पहिलोपटक मेले...",Employment
2,\n﻿काठमाडौं । नेपाल प्रहरीले नयाँ भर्ना खुलाएक...,Employment
3,"\n﻿यूएई, १४ माघ – वैदेशिक रोजगारीका लागि उत्क...",Employment
4,"\n﻿\n१६ वैशाख, काठमाडौं । ‘काम पाइएन डिग्री पा...",Employment
...,...,...
14351,"\n﻿श्रावण २९, काठमाण्डौं । भारतीय स्मार्टफोन न...",Technology
14352,\n﻿एन्टिभाइरसले कम्प्युटरको लागि एउटा सुरक्षा ...,Technology
14353,\n﻿फेसबुकले आफ्ना प्रयोगकर्ताको गेम खेल्ने आदत...,Technology
14354,"\n﻿श्रावण २१, काठमाण्डौं । युरोपका वैज्ञानिकहर...",Technology


In [10]:
import string
import os
import sys
import re

class Preprocessor:
    # def __init__(self):
        # self.this_dir, self.this_file = os.path.split(__file__)

    def clean_text(self,text):
        '''
        accepts the plain text and makes
        use of regex for cleaning the noise
        :param: text :type:str
        :return:cleaned text :type str
        '''
        text = text.replace("\n","")
        text = text.replace("\t","")
        text = text.replace("\ufeff","")
        #remove all the special characters
        symbols = "।,;?!—-."
        remove_symbols = str.maketrans('', '', symbols)
        text = text.translate(remove_symbols)
        # remove digits
        digits = "०१२३४५६७८९"
        remove_digits = str.maketrans('', '', digits)
        text = text.translate(remove_digits)
        return text
        # try:
        #     text = text.encode('ascii', errors='ignore').decode("utf-8")
        #     return text
        # except:
        #     return text
    

    def sentence_tokenize(self, text):
        """This function tokenize the sentences
        
        Arguments:
            text {string} -- Sentences you want to tokenize
        
        Returns:
            sentence {list} -- tokenized sentence in list
        """
        sentences = text.strip().split(u"।")
        sentences = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences]
        return sentences

    def word_tokenize(self, sentence, new_punctuation=[]):
        """This function tokenize with respect to word
        
        Arguments:
            sentence {string} -- sentence you want to tokenize
            new_punctuation {list} -- more punctutaion for tokenizing  default ['।',',',';','?','!','—','-']
        
        Returns:
            list -- tokenized words
        """
        punctuations = ['।', ',', ';', '?', '!', '—', '-', '.']
        if new_punctuation:
            punctuations = set(punctuations + new_punctuation)

        for punct in punctuations:
            sentence = ' '.join(sentence.split(punct))

        return sentence.split()

    def character_tokenize(self, word):
        """ Returns the tokenization in character level.
        
        Arguments:
            word {string} -- word to be tokenized in character level.
        
        Returns:
            [list] -- list of characters
        """
        try:
            import icu

        except:
            print("please install PyICU")
        
        temp_ = icu.BreakIterator.createCharacterInstance(icu.Locale())
        temp_.setText(word)
        char = []
        i = 0
        for j in temp_:
            s = word[i:j]
            char.append(s)
            i = j

        return char

In [11]:
nep_preprocessor = Preprocessor()

In [12]:
news_df['cleaned_news'] = news_df['news'].apply(nep_preprocessor.clean_text)

In [13]:
news_df

,news,labels,cleaned_news
0,"\n﻿कारोबार संवाददाता\nकाठमाडौं, ९ असार\nगैरआवा...",Employment,कारोबार संवाददाताकाठमाडौं असारगैरआवासीय नेपाल...
1,"\n﻿\n४ पुस, काठमाडौं । १८ वर्षमा पहिलोपटक मेले...",Employment,पुस काठमाडौं वर्षमा पहिलोपटक मेलेसियाली मुद...
2,\n﻿काठमाडौं । नेपाल प्रहरीले नयाँ भर्ना खुलाएक...,Employment,काठमाडौं नेपाल प्रहरीले नयाँ भर्ना खुलाएको छ ...
3,"\n﻿यूएई, १४ माघ – वैदेशिक रोजगारीका लागि उत्क...",Employment,यूएई माघ – वैदेशिक रोजगारीका लागि उत्कृष्ट ग...
4,"\n﻿\n१६ वैशाख, काठमाडौं । ‘काम पाइएन डिग्री पा...",Employment,वैशाख काठमाडौं ‘काम पाइएन डिग्री पास गरेर भै...
...,...,...,...
14351,"\n﻿श्रावण २९, काठमाण्डौं । भारतीय स्मार्टफोन न...",Technology,श्रावण काठमाण्डौं भारतीय स्मार्टफोन निर्यात ...
14352,\n﻿एन्टिभाइरसले कम्प्युटरको लागि एउटा सुरक्षा ...,Technology,एन्टिभाइरसले कम्प्युटरको लागि एउटा सुरक्षा कवच...
14353,\n﻿फेसबुकले आफ्ना प्रयोगकर्ताको गेम खेल्ने आदत...,Technology,फेसबुकले आफ्ना प्रयोगकर्ताको गेम खेल्ने आदतको ...
14354,"\n﻿श्रावण २१, काठमाण्डौं । युरोपका वैज्ञानिकहर...",Technology,श्रावण काठमाण्डौं युरोपका वैज्ञानिकहरुले एक ...


In [14]:
import os
# from gensim.models.doc2vec import Doc2Vec
# from nepali_data_preprocesser import Preprocessor
import string
import sys
import re
import pandas as pd 
import numpy
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
# from NepaliLemmatizermaster.Lemmatization.lemmatizer import return_lemma_word



# model = Doc2Vec.load("/home/info/Desktop/pudasaini/nepali_embedding/bakya2vec/model/bakya2vec_lg.model")

In [15]:
corpus = news_df['cleaned_news'].tolist()

# TFIDF

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
Tfidf_vect = vectorizer.fit_transform(corpus)


In [44]:
X,y = news_df['cleaned_news'], news_df['labels']
train_X,  test_X, train_y, test_y = train_test_split(X,y, test_size = 0.25, random_state = 520)

In [47]:
Train_X_Tfidf = vectorizer.fit_transform(train_X)
Test_X_Tfidf = vectorizer.transform(test_X)

In [49]:
from sklearn import model_selection, naive_bayes, svm
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,train_y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)


In [52]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, predictions_NB)

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   50,
           0,    0,    0,    0,    0],
       [   0,    2,    0,    0,    0,    0,    0,    0,    0,    0,  164,
           0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  102,
           0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   55,
           0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  544,
           0,    3,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   49,
           0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   74,
           0,    0,    0,    0,    0],
       [   0,    2,    0,    0,    0,    0,    0,   51,    0,    0,  542,
           0,    3,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   36,
           0,    1,   

In [54]:
from sklearn.metrics import classification_report
print(classification_report(test_y, predictions_NB))

                    precision    recall  f1-score   support

              Auto       0.00      0.00      0.00        50
              Bank       0.50      0.01      0.02       166
              Blog       0.00      0.00      0.00       102
Business Interview       0.00      0.00      0.00        55
           Economy       0.00      0.00      0.00       547
         Education       0.00      0.00      0.00        49
        Employment       0.00      0.00      0.00        74
     Entertainment       0.98      0.09      0.16       598
         Interview       0.00      0.00      0.00        37
        Literature       0.00      0.00      0.00         7
     National News       0.56      0.99      0.71      3500
           Opinion       0.00      0.00      0.00       272
            Sports       0.91      0.47      0.62      1090
        Technology       0.00      0.00      0.00        53
           Tourism       0.00      0.00      0.00       111
             World       0.00      0.00

/home/shushant/anaconda3/envs/mix/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/shushant/anaconda3/envs/mix/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/shushant/anaconda3/envs/mix/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [55]:
svc = SVC()


In [ ]:
svc.fit(Train_X_Tfidf,train_y)

In [ ]:
svc_predict = svc.predict(Test_X_Tfidf)

In [ ]:
svc_predict

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, svc_predict)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, svc_predict))

# DOC2VEC

In [189]:
def find_embedding(text):
    text = re.sub(r"[|a-zA-z.'#0-9@,:?'\u200b\u200c\u200d!/&~-]",'',text)
    tokenized_text = nep_preprocessor.word_tokenize(text)
    text_embedding = model.infer_vector(tokenized_text)
    return text_embedding

In [190]:
news_df['embeddings'] = news_df['cleaned_news'].apply(find_embedding)

In [192]:
news_df

,news,labels,cleaned_news,embeddings
0,\n﻿नायक श्रीकृष्ण श्रेष्ठकी श्रीमति अर्थात नाय...,Entertainment,नायक श्रीकृष्ण श्रेष्ठकी श्रीमति अर्थात नायिका...,"[-0.080443375, 0.2827489, 0.83980685, 0.197515..."
1,\n﻿फिल्मको पर्दामा हेरेर मन भर्नुपर्ने नायिकाह...,Entertainment,फिल्मको पर्दामा हेरेर मन भर्नुपर्ने नायिकाहरु ...,"[-0.860313, -0.71543497, 0.2471731, 0.50896996..."
2,\n﻿माउ पार्टीसँग एकता एजेन्डामा मोहन वैद्य नेत...,Entertainment,माउ पार्टीसँग एकता एजेन्डामा मोहन वैद्य नेतृत्...,"[-0.16163519, 0.0707252, 0.024961947, 0.004659..."
3,\n﻿भगवान गौतम बुद्धको जन्मस्थल लुम्बिनीमा चीनक...,Entertainment,भगवान गौतम बुद्धको जन्मस्थल लुम्बिनीमा चीनका व...,"[0.7351121, 0.09960666, 0.34827855, 0.17212139..."
4,\n﻿मकवानपुर जिल्लालाई नेपालको ३० औं साक्षर जिल...,Entertainment,मकवानपुर जिल्लालाई नेपालको औं साक्षर जिल्ला घ...,"[0.08674331, -0.22013651, 0.025190195, 0.11819..."
...,...,...,...,...
14351,"\n﻿\n७ पुस, काठमाडौं । नेपालमा लोकपि्रय बनेको ...",Auto,पुस काठमाडौं नेपालमा लोकपि्रय बनेको ‘टाटा सफ...,"[-0.6924653, 0.46029034, -0.2538125, 0.8334381..."
14352,"\n﻿\n\n१२ पुस, काठमाडाैं। वातावरण मैत्री सवारी...",Auto,पुस काठमाडाैं वातावरण मैत्री सवारी साधनको अधि...,"[-0.70128024, 0.003827243, 0.52480596, -0.4330..."
14353,"\n﻿\n२७ मंसिर, काठमाडौं । टाटा मोटर्सले ट्याक्...",Auto,मंसिर काठमाडौं टाटा मोटर्सले ट्याक्सीका लागि...,"[-0.12963124, -0.15761732, 0.45340633, -0.1884..."
14354,"\n﻿२ पुस, काठमाडौं । लामो समयदेखिको तराई आन्दो...",Auto,पुस काठमाडौं लामो समयदेखिको तराई आन्दोलन र भ...,"[0.06432456, 0.009475079, 0.2335211, -0.728735..."


In [27]:
X,y = news_df['embeddings'], news_df['labels']

KeyError: 'embeddings'

In [194]:
X = [numpy.array(i) for i in X]

In [25]:
svc = SVC()

In [26]:
train_X,  test_X, train_y, test_y = train_test_split(X,y, test_size = 0.25, random_state = 520)

NameError: name 'y' is not defined

In [197]:
svc.fit(train_X, train_y)

SVC()

In [198]:
svc_predict = svc.predict(test_X)

In [200]:
svc_predict

array(['National News', 'National News', 'National News', ...,
       'National News', 'National News', 'Opinion'], dtype=object)

In [201]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, svc_predict)

array([[  38,    0,    0,    0,    1,    0,    0,    1,    0,    0,    8,
           0,    0,    0,    0,    0],
       [   0,  111,    0,    0,   18,    0,    0,    0,    0,    0,   37,
           0,    7,    0,    0,    0],
       [   0,    0,   45,    0,    0,    1,    0,   19,    0,    0,   22,
           3,   23,    0,    0,    0],
       [   0,    0,    0,   49,    1,    0,    0,    0,    0,    0,   13,
           2,    0,    0,    0,    0],
       [   4,   12,    0,    0,  372,    0,    3,    1,    0,    0,  159,
           0,    2,    1,    2,    0],
       [   0,    0,    0,    0,    0,    1,    0,    1,    0,    0,   43,
           1,    6,    0,    0,    0],
       [   0,    2,    0,    0,    3,    0,   23,    0,    0,    0,   41,
           0,    3,    0,    0,    0],
       [   0,    1,   12,    0,    3,    1,    0,  439,    0,    0,   65,
           0,   66,    0,    0,    0],
       [   0,    0,    1,    0,    0,    0,    0,    3,   22,    0,    2,
           7,    0,   

In [206]:
from sklearn.metrics import classification_report
print(classification_report(test_y, svc_predict))

                    precision    recall  f1-score   support

              Auto       0.84      0.79      0.82        48
              Bank       0.80      0.64      0.71       173
              Blog       0.68      0.40      0.50       113
Business Interview       1.00      0.75      0.86        65
           Economy       0.79      0.67      0.72       556
         Education       0.33      0.02      0.04        52
        Employment       0.88      0.32      0.47        72
     Entertainment       0.84      0.75      0.79       587
         Interview       0.96      0.63      0.76        35
        Literature       1.00      0.50      0.67         8
     National News       0.83      0.94      0.89      3509
           Opinion       0.92      0.84      0.88       285
            Sports       0.81      0.87      0.84      1054
        Technology       0.84      0.43      0.57        60
           Tourism       0.91      0.38      0.53       106
             World       0.91      0.33